# Hyperparameter Tuning with Optuna

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thawn/ttt-workshop-cnn/blob/main/book/pair_programming/optuna_tuning.ipynb)

Note: this notebook requires a GPU. Please switch to T4 GPU using the drop down menu in the top right of the colab interface:

<img src="assets/change_runtime.png" alt="switch to GPU" width="350"/>

## Tune learning rate, weight decay, and channels on a toy training loop

In [1]:
# Dependency management
import sys, subprocess
for p in ['optuna', 'plotly', 'scikit-learn', 'torch']:
    try:
        __import__(p if p != 'scikit-learn' else 'sklearn')
    except Exception:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', p, '-q'])

In [2]:
import optuna
import torch, torch.nn as nn, torch.optim as optim
from pathlib import Path

# fix random seeds for reproducibility
torch.random.manual_seed(0)

# set device
device = "cuda" if torch.cuda.is_available() else "cpu"
device = "mps" if torch.backends.mps.is_available() else device  ## Mac OS metal
print(f"Using device: {device}")

Using device: mps


In [3]:
# Data
X = torch.randn(64,1,16,16)
y = torch.randn(64,1,16,16)

# Model
class Tiny(nn.Module):
    def __init__(self, ch=8):
        super().__init__()
        self.net = nn.Sequential(nn.Conv2d(1,ch,3,padding=1), nn.ReLU(), nn.Conv2d(ch,1,3,padding=1))
    def forward(self,x): return self.net(x)

# Loss
criterion = nn.MSELoss()


In [4]:
# Objective function
def objective(trial):
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    wd = trial.suggest_float('weight_decay', 0.0, 1e-3)
    ch = trial.suggest_categorical('channels', [4,8,16,32])
    model = Tiny(ch)
    opt = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    for _ in range(10):
        opt.zero_grad(); loss = criterion(model(X), y); loss.backward(); opt.step()
    return float(loss)

In [ ]:
# Create study
study_name = "tiny_optuna_study"

db_path = Path('data') / f"{study_name}.db"
db_path.parent.mkdir(parents=True, exist_ok=True)

study = optuna.create_study(
    direction="minimize", study_name=study_name, storage=f"sqlite:///{db_path}", load_if_exists=True
)

[I 2025-09-15 12:22:51,685] A new study created in RDB with name: tiny_optuna_study


In [6]:
# Optimize the study
study.optimize(objective, n_trials=30)
print('Best:', study.best_params)

/var/folders/8s/7jr95f1d28vflnwm1cfqv7t40000gr/T/ipykernel_75846/2863418574.py:10: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  return float(loss)
[I 2025-09-15 12:22:52,087] Trial 0 finished with value: 1.0426946878433228 and parameters: {'lr': 0.0002868739861303541, 'weight_decay': 0.00029171941759028843, 'channels': 8}. Best is trial 0 with value: 1.0426946878433228.
[I 2025-09-15 12:22:52,161] Trial 1 finished with value: 1.0603933334350586 and parameters: {'lr': 0.00018793841070340842, 'weight_decay': 0.00046201715168339084, 'channels': 4}. Best is trial 0 with value: 1.0426946878433228.
[I 2025-09-15 12:22:52,291] Trial 2 finished with value: 1.0234827995300293 and parameters: {'lr': 0.00016368997015435812, 'weight_decay': 0.0008962908502345968, 'channels

Best: {'lr': 0.0066912994039458, 'weight_decay': 0.0003765439004842885, 'channels': 4}


In [7]:
# Plotting
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

In [8]:
# Plot hyperparameter importance
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [9]:
# Plot parallel coordinates
fig = optuna.visualization.plot_parallel_coordinate(study)
fig.show()

## Exercise

Use optuna to optimize the U-Net in the [U-Net notebook](https://colab.research.google.com/github/thawn/ttt-workshop-cnn/blob/main/book/exercises/unet_bbbc039_pair_programming.ipynb). 

Do not forget to add the dependencies and then **restart the notebook**.

## Challenge

Can you beat the best manual tuning?

Which team gets the best test score using optuna?